<a href="https://colab.research.google.com/github/thaisflopes/projeto_de_pesquisa_judicializacao_transporte_aereo/blob/main/Meta_A05_Prepara%C3%A7%C3%A3o_dos_dados_para_os_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Bibliotecas
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

### Baixando bases de dados

In [ ]:
#Baixando dados pelo Google Drive
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Projeto de pesquisa/Base_Dados/Códigos Finais - Aprendizado de Máquina/dados_aereas_ML.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Baixando dados pelo Anaconda
#data = pd.read_csv('dados_aereas_ML.csv')

In [ ]:
#Contabilizando a quantidade de linhas inicial
num_linhas_inicial = data.shape[0]

print("Número de linhas inicial:", num_linhas_inicial)

Número de linhas inicial: 126041


In [ ]:
#Visualizando dados
print('\n Visualização do DataFrame:\n', data.head())
print('\n Tipos de dados do DataFrame:\n', data.dtypes)


 Visualização do DataFrame:
   empresa_re  num_proc_ano data_distribuicao  UF   operacao  \
0       Azul          2020        03/01/2020  PR  Doméstico   
1       Azul          2019        07/01/2019  SP  Doméstico   
2       Azul          2020        03/01/2020  PR  Doméstico   
3       Azul          2022        01/01/2022  TO  Doméstico   
4       Azul          2021        12/01/2021  SP  Doméstico   

               jurisdicao     instancia situacao_processual_sentenca  \
0  Juizado Especial Cível  1ª Instância                   Condenação   
1  Juizado Especial Cível  1ª Instância                   Condenação   
2  Juizado Especial Cível  2ª Instância                   Condenação   
3  Juizado Especial Cível  1ª Instância                   Condenação   
4  Juizado Especial Cível  1ª Instância                   Condenação   

   n_autores  valor_pedido_inicial  ...  obj_bagagem  obj_contrato  \
0          1               10000.0  ...        False         False   
1          2      

### Preparando as variáveis para os modelos

Alterações:
- UF: reduzir em regiões
- valor_pedido_inicial: transformar em Baixo, Médio e Alto
- valor_dano_moral: transformar em Baixo, Médio e Alto
- valor_dano_material: transformar em Baixo, Médio e Alto
- valor_total_condenacao: transformar em Baixo, Médio e Alto

Sem alterações: empresa_re, num_proc_ano, operacao, jurisdicao, instancia, situacao_processual_sentenca, n_autores, obj_problemas_operacionais, obj_bagagem, obj_contrato, obj_outros, causa_nao_informado, causa_forca_maior, causa_fortuitos, causa_terceiros, causa_cliente, causa_aerea, causa_outros

Excluir: data_distribuicao, UF, valor_pago

#### UF para Região

In [ ]:
#Criando a categoria de região
uf = [
    data['UF'].isin(['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO']),  # Norte
    data['UF'].isin(['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']),  # Nordeste
    data['UF'].isin(['GO', 'MT', 'MS', 'DF']),  # Centro-Oeste
    data['UF'].isin(['ES', 'MG', 'RJ', 'SP']),  # Sudeste
    data['UF'].isin(['PR', 'RS', 'SC'])  # Sul
]

regiao = ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']

data['regiao'] = np.select(uf, regiao, default='Outra')

data[['UF', 'regiao']].head()

,UF,regiao
0,PR,Sul
1,SP,Sudeste
2,PR,Sul
3,TO,Norte
4,SP,Sudeste


#### Excluindo colunas que não serão usadas

In [ ]:
#Excluindo colunas que não serão usadas
data = data.drop(['data_distribuicao', 'UF', 'valor_pago'], axis=1)

#### Indenizações

In [ ]:
#Removendo outliers (valores acima de 1 milhão)

#Lista de colunas para verificar os outliers
colunas_indenizacoes = ['valor_pedido_inicial', 'valor_dano_moral', 'valor_dano_material', 'valor_total_condenacao']

#Definindo valor máximo para remoção de outliers
valor_maximo = 1e6  # R$ 1.000.000

#Removendo as linhas que têm valores maiores que 1.000.000 nas colunas especificadas
data = data[~((data[colunas_indenizacoes] > valor_maximo).any(axis=1))]

#Contabilizando a quantidade de linhas após a remoção dos outliers
num_linhas_sem_outlier = data.shape[0]

print("Número de linhas após a remoção dos outliers:", num_linhas_sem_outlier)

Número de linhas após a remoção dos outliers: 126036


##### Dano Moral

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_moral'
percentil_25_dano_moral = data['valor_dano_moral'].quantile(0.25)
percentil_50_dano_moral = data['valor_dano_moral'].quantile(0.50)
percentil_75_dano_moral = data['valor_dano_moral'].quantile(0.75)

print(percentil_25_dano_moral, percentil_50_dano_moral, percentil_75_dano_moral)

0.0 0.0 4000.0


In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_moral' somente com os valores únicos

#Obtendo os valores únicos da coluna 'valor_dano_moral'
valores_unicos_dano_moral = data['valor_dano_moral'].dropna().unique()

#Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_dano_moral = np.min(valores_unicos_dano_moral)
percentil_25_dano_moral = np.percentile(valores_unicos_dano_moral, 25)
percentil_50_dano_moral = np.percentile(valores_unicos_dano_moral, 50)
percentil_75_dano_moral = np.percentile(valores_unicos_dano_moral, 75)
valor_maximo_dano_moral = np.max(valores_unicos_dano_moral)

#Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_dano_moral}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_dano_moral}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_dano_moral}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_dano_moral}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_dano_moral}")


Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 1489.0575000000001
Percentil 50 (com base nos valores únicos): 3227.9449999999997
Percentil 75 (com base nos valores únicos): 5953.0075
Máximo (com base nos valores únicos): 465000.0


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_dano_moral']),  # Nulos
    data['valor_dano_moral'] <= percentil_25_dano_moral,  # Baixo: até o percentil 25
    (data['valor_dano_moral'] > percentil_25_dano_moral) & (data['valor_dano_moral'] <= percentil_75_dano_moral),  # Médio: entre o percentil 25 e 75
    data['valor_dano_moral'] > percentil_75_dano_moral  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Criar a nova coluna 'valor_dano_moral_novo' com as categorias
data['valor_dano_moral_novo'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_dano_moral_novo'
data['valor_dano_moral_novo'].value_counts()

,count
valor_dano_moral_novo,
Baixo,69298
Médio,38662
Alto,18076


In [ ]:
#Criando arquivo csv (valores de input de indenizações numéricos)
data.to_csv('dados_modelos_ind_num.csv', index=False, encoding='utf-8')

#### Valor pedido inicial

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_pedido_inicial'
percentil_25_pedido_inicial = data['valor_pedido_inicial'].quantile(0.25)
percentil_50_pedido_inicial = data['valor_pedido_inicial'].quantile(0.50)
percentil_75_pedido_inicial = data['valor_pedido_inicial'].quantile(0.75)

print(percentil_25_pedido_inicial, percentil_50_pedido_inicial, percentil_75_pedido_inicial)

10000.0 15000.0 21273.6725


In [ ]:
#Obtendo os valores únicos da coluna 'valor_pedido_inicial'
valores_unicos_pedido_inicial = data['valor_pedido_inicial'].dropna().unique()

#Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_pedido_inicial = np.min(valores_unicos_pedido_inicial)
percentil_25_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 25)
percentil_50_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 50)
percentil_75_pedido_inicial = np.percentile(valores_unicos_pedido_inicial, 75)
valor_maximo_pedido_inicial = np.max(valores_unicos_pedido_inicial)

#Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_pedido_inicial}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_pedido_inicial}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_pedido_inicial}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_pedido_inicial}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_pedido_inicial}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 10372.59
Percentil 50 (com base nos valores únicos): 15420.44
Percentil 75 (com base nos valores únicos): 22526.68
Máximo (com base nos valores únicos): 950000.0


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_pedido_inicial']),  # Nulos
    data['valor_pedido_inicial'] <= percentil_25_pedido_inicial,  # Baixo: até o percentil 25
    (data['valor_pedido_inicial'] > percentil_25_pedido_inicial) & (data['valor_pedido_inicial'] <= percentil_75_pedido_inicial),  # Médio: entre o percentil 25 e 75
    data['valor_pedido_inicial'] > percentil_75_pedido_inicial  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Criar a nova coluna 'valor_pedido_inicial_novo' com as categorias
data['valor_pedido_inicial_novo'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_pedido_inicial_novo'
data['valor_pedido_inicial_novo'].value_counts()

,count
valor_pedido_inicial_novo,
Médio,59146
Baixo,38556
Alto,28334


#### Dano Material

In [ ]:
#Calculando os percentis 25, 50 e 75 da coluna 'valor_dano_material'
percentil_25_dano_material = data['valor_dano_material'].quantile(0.25)
percentil_50_dano_material = data['valor_dano_material'].quantile(0.50)
percentil_75_dano_material = data['valor_dano_material'].quantile(0.75)

print(percentil_25_dano_material, percentil_50_dano_material, percentil_75_dano_material)

0.0 0.0 192.0


In [ ]:
# Obtendo os valores únicos da coluna 'valor_dano_material'
valores_unicos_dano_material = data['valor_dano_material'].dropna().unique()

# Calculando os percentis 25, 50 e 75 dos valores únicos
valor_minimo_dano_material = np.min(valores_unicos_dano_material)
percentil_25_dano_material = np.percentile(valores_unicos_dano_material, 25)
percentil_50_dano_material = np.percentile(valores_unicos_dano_material, 50)
percentil_75_dano_material = np.percentile(valores_unicos_dano_material, 75)
valor_maximo_dano_material = np.max(valores_unicos_dano_material)

# Exibindo os percentis e os valores mínimos e máximos
print(f"Mínimo (com base nos valores únicos): {valor_minimo_dano_material}")
print(f"Percentil 25 (com base nos valores únicos): {percentil_25_dano_material}")
print(f"Percentil 50 (com base nos valores únicos): {percentil_50_dano_material}")
print(f"Percentil 75 (com base nos valores únicos): {percentil_75_dano_material}")
print(f"Máximo (com base nos valores únicos): {valor_maximo_dano_material}")

Mínimo (com base nos valores únicos): 0.0
Percentil 25 (com base nos valores únicos): 447.955
Percentil 50 (com base nos valores únicos): 1097.17
Percentil 75 (com base nos valores únicos): 2601.45
Máximo (com base nos valores únicos): 453880.56


In [ ]:
# Definindo as condições para as categorias com base nos percentis calculados

condicoes = [
    pd.isnull(data['valor_dano_material']),  # Nulos
    data['valor_dano_material'] <= percentil_25_dano_material,  # Baixo: até o percentil 25
    (data['valor_dano_material'] > percentil_25_dano_material) & (data['valor_dano_material'] <= percentil_75_dano_material),  # Médio: entre o percentil 25 e 75
    data['valor_dano_material'] > percentil_75_dano_material  # Alto: acima do percentil 75
]

# Definir as opções para as categorias
nulos = None
opcoes = [nulos, 'Baixo', 'Médio', 'Alto']

# Criar a nova coluna 'valor_dano_material_novo' com as categorias
data['valor_dano_material_novo'] = np.select(condicoes, opcoes)

# Contar a frequência dos valores na coluna 'valor_dano_material_novo'
data['valor_dano_material_novo'].value_counts()


,count
valor_dano_material_novo,
Baixo,100595
Médio,17270
Alto,8171


In [ ]:
#Excluindo colunas de indenizações numéricas
data = data.drop(['valor_pedido_inicial', 'valor_dano_moral', 'valor_dano_material', 'valor_total_condenacao'], axis=1)

print(data.columns)

Index(['empresa_re', 'num_proc_ano', 'operacao', 'jurisdicao', 'instancia',
       'situacao_processual_sentenca', 'n_autores',
       'obj_problemas_operacionais', 'obj_bagagem', 'obj_contrato',
       'obj_outros', 'causa_nao_informado', 'causa_forca_maior',
       'causa_fortuitos', 'causa_terceiros', 'causa_cliente', 'causa_aerea',
       'causa_outros', 'regiao', 'valor_dano_moral_novo',
       'valor_pedido_inicial_novo', 'valor_dano_material_novo'],
      dtype='object')


In [ ]:
#Criando arquivo csv
data.to_csv('dados_modelos_ind_cat.csv', index=False, encoding='utf-8')